In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
import os
import pandas as pd

PROJECT_ROOT = Path.cwd().parents[0]
SRC_PATH = PROJECT_ROOT / "src"
sys.path.append(str(SRC_PATH))

from MicroNAV.adapters.outbound.db.sqlite_db import Db_datos


from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]
DB_PATH = PROJECT_ROOT / "data" / "microvap.db"
DATA_RAW = PROJECT_ROOT / "data" / "raw"

db_data = Db_datos(DB_PATH)


/home/versekert/Documentos/Ucts/MicroNAV/data/microvap.db


In [3]:
df = pd.read_excel(DATA_RAW / "microvap_metadata_complete_15012026.xlsx")
df.columns = df.columns.str.strip()

## Created Primary keys 
df['admission_date_icu'] = pd.to_datetime(
    df['admission_date_icu']
).dt.strftime('%Y-%m-%d')
df['admission_id'] = (
    df['patient_id'].astype(str)
    + "_"
    + df['admission_date_icu']
)


df.head()

/tmp/ipykernel_77261/2998006173.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['admission_id'] = (


,specimen,copies_ul_dna,patient_id,timepoint,timepoint_character,tissue_type,admission_date_cus,admission_date_icu,days_stay_icu,pneumonia,...,Otu16185,Otu16895,Otu17366,Otu17509,Otu17573,Otu17638,Otu18078,Otu18302,Otu18444,admission_id
0,006_1_BAL,3459.212646,6,uno,baseline,bal,2020-01-28,2020-01-28,5,no_vap,...,0.0,0,0.0,0,0,0.0,0,0,0,6_2020-01-28
1,006_1_NASAL,NaN,6,uno,baseline,nasal,2020-01-28,2020-01-28,5,no_vap,...,0.0,0,0.0,0,0,0.0,0,0,0,6_2020-01-28
2,006_1_TS,14486.053060,6,uno,baseline,ts,2020-01-28,2020-01-28,5,no_vap,...,0.0,0,0.0,0,0,0.0,0,0,0,6_2020-01-28
3,011_1_BAL,804.843343,11,uno,baseline,bal,2020-02-05,2020-02-05,12,no_vap,...,0.0,0,0.0,0,0,0.0,0,0,0,11_2020-02-05
4,011_1_NASAL,NaN,11,uno,baseline,nasal,2020-02-05,2020-02-05,12,no_vap,...,0.0,0,0.0,0,0,0.0,0,0,0,11_2020-02-05


## Catalog Created

### OTU

In [4]:
df_otuCatalog = pd.read_excel(DATA_RAW / "OtuCatalog.xlsx")

n = db_data.load_df(df_otuCatalog, "dim_otu", clear_table=True)
print(f"Insertados: {n}")


Insertados: 2096


### Antibiotic

In [5]:
df_antibiotic_Catalog = pd.read_excel(DATA_RAW / "AntibioticCatalog.xlsx")


df_antibiotic_Catalog['drug_name'] = df_antibiotic_Catalog['drug_name'].astype(str).str.strip()
df_antibiotic_Catalog = df_antibiotic_Catalog.dropna(subset=['drug_name']).drop_duplicates(subset=['drug_name'])

n = db_data.load_df(df_antibiotic_Catalog, "dim_antibiotic", clear_table=True)
print(f"Insertados: {n}")

Insertados: 24


## Update table Patient

In [6]:
variables_list= ['patient_id',
                'gender',
                'height',
                'weight',
                'alcoholism',
                'anemia',
                'stroke',
                'asthma',
                'cancer',
                'diabetes',
                'coronary_heart_disease',
                'mental_illness',
                'chronic_kidney_disease',
                'copd',
                'heart_failure',
                'hemodialysis',
                'hypertension',
                'ppi_chronic',
                'obesity',
                'sahos',
                'smoking',
                'statin',
                'no_background']

df_patient = df[variables_list]
df_patient.head()

,patient_id,gender,height,weight,alcoholism,anemia,stroke,asthma,cancer,diabetes,...,copd,heart_failure,hemodialysis,hypertension,ppi_chronic,obesity,sahos,smoking,statin,no_background
0,6,1,163.0,70.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,6,1,163.0,70.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,6,1,163.0,70.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,11,1,167.0,60.0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,11,1,167.0,60.0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [7]:
df_grouped = (
    df_patient
    .groupby('patient_id', as_index=False)
    .agg({
        'gender': 'first',
        'height': 'max',
        'weight': 'max',
        'alcoholism': 'max',
        'anemia': 'max',
        'stroke': 'max',
        'asthma': 'max',
        'cancer': 'max',
        'diabetes': 'max',
        'coronary_heart_disease': 'max',
        'mental_illness': 'max',
        'chronic_kidney_disease': 'max',
        'copd': 'max',
        'heart_failure': 'max',
        'hemodialysis': 'max',
        'hypertension': 'max',
        'ppi_chronic': 'max',
        'obesity': 'max',
        'sahos': 'max',
        'smoking': 'max',
        'statin': 'max',
        'no_background': 'max'
    })
)

In [8]:
df_grouped[df_grouped.duplicated(subset='patient_id', keep=False)] \
    .sort_values('patient_id')

,patient_id,gender,height,weight,alcoholism,anemia,stroke,asthma,cancer,diabetes,...,copd,heart_failure,hemodialysis,hypertension,ppi_chronic,obesity,sahos,smoking,statin,no_background


In [9]:
n = db_data.load_df(df_grouped, "patient", clear_table=True)
print(f"Insertados: {n}")

IntegrityError: FOREIGN KEY constraint failed

## Update table Admission

In [ ]:
variables_list= [
                'patient_id',
                'admission_id',
                'admission_infectious',
                'admission_general_surgery',
                'admission_cardiovascular',
                'admission_trauma',
                'admission_nephrological',
                'admission_neurological',
                'admission_neurosurgical',
                'admission_autoimmune',
                'admission_toxic',
                'admission_metabolic',
                'other_diagnostic_admission']

df_admission = df[variables_list]

df_admission.head()

,patient_id,admission_id,admission_infectious,admission_general_surgery,admission_cardiovascular,admission_trauma,admission_nephrological,admission_neurological,admission_neurosurgical,admission_autoimmune,admission_toxic,admission_metabolic,other_diagnostic_admission
0,6,6_2020-01-28,0,0,0,1,0,0,0,0,0,0,0
1,6,6_2020-01-28,0,0,0,1,0,0,0,0,0,0,0
2,6,6_2020-01-28,0,0,0,1,0,0,0,0,0,0,0
3,11,11_2020-02-05,0,0,1,0,0,0,0,0,0,0,0
4,11,11_2020-02-05,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
df_grouped = (
    df_admission
    .groupby('admission_id', as_index=False)
    .agg({
        'patient_id': 'first',
        'admission_id': 'max',
        'admission_infectious': 'max',
        'admission_general_surgery': 'max',
        'admission_cardiovascular': 'max',
        'admission_trauma': 'max',
        'admission_nephrological': 'max',
        'admission_neurological': 'max',
        'admission_neurosurgical': 'max',
        'admission_autoimmune': 'max',
        'admission_toxic': 'max',
        'admission_metabolic': 'max',
        'other_diagnostic_admission': 'max'
    })
)

In [ ]:
df_grouped[df_grouped.duplicated(subset='patient_id', keep=False)] \
    .sort_values('patient_id')

,patient_id,admission_id,admission_infectious,admission_general_surgery,admission_cardiovascular,admission_trauma,admission_nephrological,admission_neurological,admission_neurosurgical,admission_autoimmune,admission_toxic,admission_metabolic,other_diagnostic_admission


In [ ]:
n = db_data.load_df(df_grouped, "admission", clear_table=True)
print(f"Insertados: {n}")

Insertados: 140


#

## Update table Lab_Results

In [ ]:
variables_list= [
            'admission_id',
            #'measured_at', --Da
            'white_blood_cells',
            'neutrophils',
            'hemoglobin',
            'hematocrit',
            'platelets',
            #'creatinine',
            'uremic_nitrogen',
            'glycemia',
            'sodium',
            'potassium',
            'total_bilirubin',
            'direct_bilirubin',
            'indirect_bilirubin',
            'alanine_gpt',
            'aspartate_got',
            'pcr',
            'procalcitonin',
            'pt',
            'ptt']

df_lab_results = df[variables_list]

df_lab_results.head()

,admission_id,white_blood_cells,neutrophils,hemoglobin,hematocrit,platelets,uremic_nitrogen,glycemia,sodium,potassium,total_bilirubin,direct_bilirubin,indirect_bilirubin,alanine_gpt,aspartate_got,pcr,procalcitonin,pt,ptt
0,6_2020-01-28,10.2,42.0,14.9,44.0,260.0,17.0,160.0,139.0,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,28.0
1,6_2020-01-28,10.2,42.0,14.9,44.0,260.0,17.0,160.0,139.0,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,28.0
2,6_2020-01-28,10.2,42.0,14.9,44.0,260.0,17.0,160.0,139.0,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,28.0
3,11_2020-02-05,9.2,84.0,8.6,25.0,120.0,29.0,100.0,141.0,4.2,0.4,0.2,0.3,17.0,30.0,NaN,NaN,16.0,28.0
4,11_2020-02-05,9.2,84.0,8.6,25.0,120.0,29.0,100.0,141.0,4.2,0.4,0.2,0.3,17.0,30.0,NaN,NaN,16.0,28.0
